# STA 141B Lecture 15

The class website is <https://github.com/2019-winter-ucdavis-sta141b/notes>

### Announcements

* Assignment 3 regrade still in progress
* Assignment 5 posted
* Starting this week, the discussion sections will run as office hours
* Project presentations

### Topics

* Databases & SQL

### Datasets

* The [Suppliers Database](http://nick-ulle.github.io/teach/suppliers.sqlite)
* The [Stock Market Database](http://anson.ucdavis.edu/~clarkf/sql/stocks.sqlite)

### References

* [W3 Schools SQL Tutorial](https://www.w3schools.com/sql/)
* [SQL Cheatsheet](http://anson.ucdavis.edu/~clarkf/sql/sql_cheatsheet.pdf)

[PDSH]: https://jakevdp.github.io/PythonDataScienceHandbook/
[ProGit]: https://git-scm.com/book/
[nlpp]: https://www.nltk.org/book/
[atap]: https://search.library.ucdavis.edu/primo-explore/fulldisplay?docid=01UCD_ALMA51320822340003126&context=L&vid=01UCD_V1&search_scope=everything_scope&tab=default_tab&lang=en_US

In [1]:
import numpy as np
import pandas as pd

import sqlite3 as sql

In [2]:
db = sql.connect("../data/suppliers.sqlite")

### Functions & Aggregation

SQL has built-in functions, which vary from one DBMS to another. The SQL cheatsheet lists most of the functions supported by SQLite.

Most SQL functions aggregate data in a column, summarizing that column somehow.

In [3]:
pd.read_sql("SELECT COUNT(*) FROM parts;", db)

,COUNT(*)
0,6


In [5]:
pd.read_sql("SELECT UPPER(City), * FROM parts LIMIT 3;", db)

,UPPER(City),PartID,PartName,Color,Weight,City
0,LONDON,1,Nut,Red,12.0,London
1,PARIS,2,Bolt,Green,17.0,Paris
2,OSLO,3,Screw,Blue,17.0,Oslo


### `GROUP BY`

The `GROUP BY` keyword groups rows before they are aggregated. `GROUP BY` is the SQL equivalent of Pandas' `.groupby()` method.

In [6]:
pd.read_sql("SELECT AVG(weight) FROM parts;", db)

,AVG(weight)
0,15.166667


In [8]:
pd.read_sql("SELECT AVG(weight), city FROM parts GROUP BY city;", db)

,AVG(weight),City
0,15.0,London
1,17.0,Oslo
2,14.5,Paris


In [10]:
pd.read_sql("SELECT AVG(weight), city, partname FROM parts GROUP BY city, partname;", db)

,AVG(weight),City,PartName
0,19.0,London,Cog
1,12.0,London,Nut
2,14.0,London,Screw
3,17.0,Oslo,Screw
4,17.0,Paris,Bolt
5,12.0,Paris,Cam


### `HAVING`

The `WHERE` keyword puts conditions on the rows returned _before computing any aggregate functions._ So use `WHERE` to remove rows before aggregation.

The `HAVING` keyword puts conditions on the rows returned _after computing any aggregate functions._ So use `HAVING` to remove rows after aggregation. `HAVING` is most useful for conditions on the result of an aggregate.

In [14]:
# It's an error to use WHERE to put a condition on an aggregated column.
pd.read_sql("SELECT AVG(weight) AS avg_weight, city FROM parts GROUP BY city WHERE avg_weight > 15", db)

DatabaseError: Execution failed on sql 'SELECT AVG(weight) AS avg_weight, city FROM parts GROUP BY city WHERE avg_weight > 15': near "WHERE": syntax error

In [15]:
pd.read_sql("SELECT AVG(weight) AS avg_weight, city FROM parts GROUP BY city HAVING avg_weight > 15", db)

,avg_weight,City
0,17.0,Oslo


### Joins

A `JOIN` combines two tables using a column they have in common. `JOIN` is the SQL equivalent of Pandas' `.join()` method and `pd.merge()` function.

In [25]:
pd.read_sql("SELECT * FROM supplierparts", db)

,PartID,SupplierID,Qty
0,1,1,300
1,1,2,200
2,1,3,400
3,1,4,200
4,1,5,100
5,1,6,100
6,2,1,300
7,2,2,400
8,3,2,200
9,4,2,200


In [19]:
pd.read_sql("""
    SELECT * FROM
    supplierparts
    LEFT JOIN
    parts
    ON supplierparts.partid = parts.partid
""", db)

,PartID,SupplierID,Qty,PartID,PartName,Color,Weight,City
0,1,1,300,1,Nut,Red,12.0,London
1,1,2,200,1,Nut,Red,12.0,London
2,1,3,400,1,Nut,Red,12.0,London
3,1,4,200,1,Nut,Red,12.0,London
4,1,5,100,1,Nut,Red,12.0,London
5,1,6,100,1,Nut,Red,12.0,London
6,2,1,300,2,Bolt,Green,17.0,Paris
7,2,2,400,2,Bolt,Green,17.0,Paris
8,3,2,200,3,Screw,Blue,17.0,Oslo
9,4,2,200,4,Screw,Red,14.0,London


SQLite supports two kinds of joins:
* Left join (`LEFT JOIN`): Keep all rows from the left table, merging rows from the right.
* Inner join (`INNER JOIN`): Keep only rows where there is a match.

Other DBMSes sometimes support two more kinds of joins:
* Right join: Keep all rows from the right table, merging rows from the left.
* Full join: Keep all rows from both tables, merging where there are matches.

In [20]:
pd.read_sql("SELECT * FROM suppliers", db)

,SupplierID,SupplierName,Status,City
0,1,Smith,20,London
1,2,Jones,10,Paris
2,3,Blake,30,Paris
3,4,Clark,20,London
4,5,Adams,30,Athens


In [21]:
pd.read_sql("""
    SELECT * FROM
    supplierparts AS l
    INNER JOIN
    suppliers AS r
    ON l.supplierid = r.supplierid;
""", db)

,PartID,SupplierID,Qty,SupplierID,SupplierName,Status,City
0,1,1,300,1,Smith,20,London
1,1,2,200,2,Jones,10,Paris
2,1,3,400,3,Blake,30,Paris
3,1,4,200,4,Clark,20,London
4,1,5,100,5,Adams,30,Athens
5,2,1,300,1,Smith,20,London
6,2,2,400,2,Jones,10,Paris
7,3,2,200,2,Jones,10,Paris
8,4,2,200,2,Jones,10,Paris
9,4,4,300,4,Clark,20,London


In [23]:
pd.read_sql("""
    SELECT l.qty, r.* FROM
    supplierparts AS l
    LEFT JOIN
    suppliers AS r
    ON l.supplierid = r.supplierid;
""", db)

,Qty,SupplierID,SupplierName,Status,City
0,300,1.0,Smith,20.0,London
1,200,2.0,Jones,10.0,Paris
2,400,3.0,Blake,30.0,Paris
3,200,4.0,Clark,20.0,London
4,100,5.0,Adams,30.0,Athens
5,100,NaN,None,NaN,None
6,300,1.0,Smith,20.0,London
7,400,2.0,Jones,10.0,Paris
8,200,2.0,Jones,10.0,Paris
9,200,2.0,Jones,10.0,Paris


### Subqueries

You can write one or more `SELECT` queries within another:

In [29]:
pd.read_sql("""
    SELECT * FROM
        (SELECT l.*, r.supplierid, r.qty FROM
        parts AS l
        INNER JOIN
        supplierparts AS r
        ON l.partid = r.partid) AS a
    INNER JOIN
        (SELECT * FROM suppliers
        WHERE status > 10) AS b
    ON a.supplierid = b.supplierid;
""", db)

,PartID,PartName,Color,Weight,City,supplierid,qty,SupplierID,SupplierName,Status,City
0,1,Nut,Red,12.0,London,1,300,1,Smith,20,London
1,1,Nut,Red,12.0,London,3,400,3,Blake,30,Paris
2,1,Nut,Red,12.0,London,4,200,4,Clark,20,London
3,1,Nut,Red,12.0,London,5,100,5,Adams,30,Athens
4,2,Bolt,Green,17.0,Paris,1,300,1,Smith,20,London
5,4,Screw,Red,14.0,London,4,300,4,Clark,20,London
6,4,Screw,Red,14.0,London,5,400,5,Adams,30,Athens


In [40]:
# You can paste strings into SQL queries, but make sure the string you paste in isn't
# compromised (e.g., user input)
val = pd.read_sql("SELECT AVG(weight) FROM parts", db).iloc[0, 0]
query = "SELECT * FROM parts WHERE weight > {}".format(val)
pd.read_sql(query, db)

,PartID,PartName,Color,Weight,City
0,2,Bolt,Green,17.0,Paris
1,3,Screw,Blue,17.0,Oslo
2,6,Cog,Red,19.0,London


In [33]:
pd.read_sql("""
    SELECT * FROM parts WHERE weight > (
        SELECT AVG(weight) FROM parts
    )
""", db)

,PartID,PartName,Color,Weight,City
0,2,Bolt,Green,17.0,Paris
1,3,Screw,Blue,17.0,Oslo
2,6,Cog,Red,19.0,London


### Database Operations: CRUD

So far we've only learned 1/4 of the basic operations most databases support. They are:

* Create (`CREATE`): Create a new table.
* Read (`SELECT`): Read some rows from a table.
* Update (`UPDATE`): Update rows in a table.
* Delete (`DROP`): Delete a table.

For data analysis, you'll usually only need to read data.

Once in a while, it can be useful to create tables to store complicated results. We have to go back to __sqlite3__ to run SQL commands other than `SELECT`.

In [66]:
result = db.execute("""
    CREATE TEMPORARY TABLE table2 AS
        SELECT l.*, r.qty, r.supplierid FROM
            parts AS l
        INNER JOIN
            supplierparts AS r
        ON l.partid = r.partid
""")

# Force the database to run ("commit") the query.
# Otherwise the database evaluates queries lazily, so the table will not show up.
db.commit()

In [69]:
pd.read_sql("SELECT * FROM table2", db)

,PartID,PartName,Color,Weight,City,Qty,SupplierID
0,1,Nut,Red,12.0,London,300,1
1,1,Nut,Red,12.0,London,200,2
2,1,Nut,Red,12.0,London,400,3
3,1,Nut,Red,12.0,London,200,4
4,1,Nut,Red,12.0,London,100,5
5,1,Nut,Red,12.0,London,100,6
6,2,Bolt,Green,17.0,Paris,300,1
7,2,Bolt,Green,17.0,Paris,400,2
8,3,Screw,Blue,17.0,Oslo,200,2
9,4,Screw,Red,14.0,London,200,2


In [68]:
pd.read_sql("SELECT * FROM sqlite_master", db)

,type,name,tbl_name,rootpage,sql
0,table,Suppliers,Suppliers,2,CREATE TABLE Suppliers (\n SupplierID integer...
1,table,Parts,Parts,3,"CREATE TABLE Parts (\n PartID integer,\n Par..."
2,table,SupplierParts,SupplierParts,4,CREATE TABLE SupplierParts (\n PartID integer...
3,index,sqlite_autoindex_SupplierParts_1,SupplierParts,5,None


In [64]:
db.execute("DROP TABLE my_table;")

db.commit()

## Other Databases

You can use the __sqlalchemy__ package to connect to many different kinds of databases (not only SQLite).

The `sqlalchemy.create_engine()` function creates an _engine_ to connect to a database. The `.raw_connection()` method on the returned engine opens a connection compatible with Pandas.

See the __sqlalchemy__ documentation for more info about [how to write a database URL](https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls).

In [70]:
import sqlalchemy

engine = sqlalchemy.create_engine("sqlite:///../data/suppliers.sqlite")
conn = engine.raw_connection()

In [71]:
pd.read_sql("SELECT * FROM sqlite_master", conn)

,type,name,tbl_name,rootpage,sql
0,table,Suppliers,Suppliers,2,CREATE TABLE Suppliers (\n SupplierID integer...
1,table,Parts,Parts,3,"CREATE TABLE Parts (\n PartID integer,\n Par..."
2,table,SupplierParts,SupplierParts,4,CREATE TABLE SupplierParts (\n PartID integer...
3,index,sqlite_autoindex_SupplierParts_1,SupplierParts,5,None


In [73]:
engine.table_names()

['Parts', 'SupplierParts', 'Suppliers']

In [72]:
conn.close()

Unlike SQLite, most databases:

* Are not stored in a single file.
* Have a client-server design, where you connect to the database by logging in with a username and password.